# Santander Customer Satisfaction 



We are given a dataset from Santander Bank, which indicates whether if a customer is satisfied or dissatisfied with their banking experience. This is based on hundreds of anonymized features. 

Installing necessary libraries :

In [ ]:
!pip install pandas-profiling numpy matplotlib seaborn --quiet

In [ ]:
!pip install jovian opendatasets xgboost graphviz lightgbm scikit-learn xgboost lightgbm --upgrade --quiet

     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 192.9 MB 63 kB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 2.0 MB 44.1 MB/s 


# Downloading dataset from Kaggle :

We can download the dataset from Kaggle directly within the Jupyter notebook using the `opendatasets` library.

In [ ]:
import os
import opendatasets as od
import pandas as pd
pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 120)
import numpy as np

In [ ]:
od.download('https://www.kaggle.com/competitions/santander-customer-satisfaction')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav246
Your Kaggle Key: ··········


100%|██████████| 8.25M/8.25M [00:00<00:00, 106MB/s]


Extracting archive ./santander-customer-satisfaction/santander-customer-satisfaction.zip to ./santander-customer-satisfaction


Let's load the data into Pandas dataframes.


In [ ]:
train_df=pd.read_csv('/content/santander-customer-satisfaction/train.csv')
test_df=pd.read_csv('/content/santander-customer-satisfaction/test.csv')
bench_df=pd.read_csv('/content/santander-customer-satisfaction/sample_submission.csv')

This is our training data, it shows all the features (columns) that we need to use to predict customer satisfaction. The "TARGET" column is the variable to predict. It equals '1' for unsatisfied customers and '0' for satisfied customers.

In [ ]:
train_df


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,88.89,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.00,0.18,3.00,2.07,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,27,3,18,48,0,186.09,0.00,91.56,138.84,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,3,6,3,0,0,0,0,0,0,0,0,0,0,0,0,

Similarly, this is the testing data

In [ ]:
test_df

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,2.43,6.00,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.100000
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,6,0,3.0,2.55,3.00,2.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.720000
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,0,9,0,90.0,57.00,51.45,66.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.950000
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.610000
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.0,3.87,30.00,21.30,0

This dataframe represents our submission format in Kaggle. It is just required to store our final predictions

In [ ]:
bench_df

,ID,TARGET
0,2,0
1,5,0
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


# Pre-processing the Data:

## 1. Checking for missing values :

We start by checking if there are any missing values in the training data by using the `isna()` function

In [ ]:
for x in list(train_df.columns) :
  if train_df[x].isna().sum()!=0:
    print('Missing values found in {}'.format(train_df.x))
  else:
    print('No missing values in any column')

No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missi

Similarly for testing data as well :

In [ ]:
for x in list(test_df.columns) :
  if test_df[x].isna().sum()!=0:
    print('Missing values found in {}'.format(test_df.x))
  else:
    print('No missing values in any column')

No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missing values in any column
No missi

Since there are no missing values, the next step is to identify numeric and categorical columns, and encode the latter. This is because we can only train a model based on numbers, not based on characters. Therefore categorical columns are encoded.

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


As we can see, there are no categorical columns, there are only numeric columns of type int and float.

We can also drop the 'ID' column since it doesn't provide any insight in the data.

In [ ]:
train_df.drop('ID',inplace=True,axis=1)
test_df.drop('ID',inplace=True,axis=1)

## 2. Using MinMax scaling to normalize the values between 0 and 1:

The next step is to normalize our data. What this does is scale all the values in the dataframe between 0 and 1. We do this to ensure a normal distribution, and to center our data. This ensure heavy weights do not have more control in the updation of gradients or lower weights having almost no control in the same.

We do this by using the `MinMaxScaler` from SciKitLearn





In [ ]:
numeric_cols = list(test_df.columns)

### Finalizing training data :

The last thing that's left is to scale the numeric features.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler().fit(train_df[numeric_cols])

In [ ]:
train_df[numeric_cols] = scaler.transform(train_df[numeric_cols])
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

In [ ]:
test_df['TARGET']=''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [ ]:
target_col='TARGET'
train_target=train_df[target_col]
test_target=test_df[target_col]

Now that the data is clean and ready to use, we are going to be using K-Fold Cross Validation technique for validation.

### K Fold Cross Validation:

K-Fold Cross Validation is a technique used in machine learning that is slightly different from classic validation. It creates a split in the training data according to the parameter we provide, and uses each different slice of the split as validation in a different cycle ([source](https://vitalflux.com/k-fold-cross-validation-python-example/)) :

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
kfold = KFold(n_splits=5)

We are defining a function `train_and_evaluate()` which takes as input - the trainig data, training targets, validation data and validation targets and returns the updated model,training and validation accuracy.

In [ ]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, input_model,**params):
    model = input_model
    model.fit(X_train, train_targets)
    train_acc = accuracy_score(model.predict(X_train),train_targets)
    val_acc = accuracy_score(model.predict(X_val), val_targets)
    return model, train_acc, val_acc

## Building Various Models for prediction :

### 1. Logistic Regression

We start by using Logistic Regression, and we do so by importing the `LogisticRegression` instance from SciKitLearn

In [ ]:
from sklearn.linear_model import LogisticRegression

Initializing the model :

In [ ]:
model_log = LogisticRegression(solver='liblinear')


The list `logistic_models` stores each model in the K-Fold technique, i.e, each model with different validation data. The first 2 lines of the `for` loop is used for obtaining training and validation data from the kfold split. We then pass this into the `train_and_evaluate` function defined previously to obtain accuaracy. Finally we are printing each set of training and validation accuracy, and the average accuracy of all splits at the end.

In [ ]:
logistic_models = []
train_avg=[]
val_avg=[]
for train_idxs, val_idxs in kfold.split(train_df[numeric_cols]):
    X_train, train_targets = train_df[numeric_cols].iloc[train_idxs], train_df[target_col].iloc[train_idxs]
    X_val, val_targets = train_df[numeric_cols].iloc[val_idxs], train_df[target_col].iloc[val_idxs]
    model, train_rmse, val_rmse = train_and_evaluate(X_train, 
                                                     train_targets, 
                                                     X_val, 
                                                     val_targets,model_log
                                                     )
    logistic_models.append(model)
    train_avg.append(train_rmse)
    val_avg.append(val_rmse)
    print('Train Accuracy: {}, Validation Accuracy: {}'.format(train_rmse, val_rmse))
print(f'Avg. Train Accuracy : {100*np.mean(train_avg):.3f} % , Avg. Val Accuracy : {100*np.mean(val_avg):.3f} %')

Train Accuracy: 0.960454485661668, Validation Accuracy: 0.9598132070507761
Train Accuracy: 0.9605367008681925, Validation Accuracy: 0.9593528018942383
Train Accuracy: 0.9591883714811892, Validation Accuracy: 0.9644172586161537
Train Accuracy: 0.960043409629045, Validation Accuracy: 0.9613917390160485
Train Accuracy: 0.9612437516443041, Validation Accuracy: 0.9564588266245725
Avg. Train Accuracy : 96.029 % , Avg. Val Accuracy : 96.029 %


The function `predict_avg` returns the average of all model predictions for the best regularized result. It takes a list of models and the testing data as input and returns the average prediction.

In [ ]:
import numpy as np

def predict_avg(models, inputs):
    return np.mean([model.predict(inputs) for model in models], axis=0)

In [ ]:
preds_log = predict_avg(logistic_models, test_df[numeric_cols])

The Accuracy reported by Logistic Regression is shown above. Let us try further with other methods.

### 2. Decision Tree

We try to improve further by using Decision Trees. It is imported similarly from SciKitLearn.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree,export_text

By hyperparameter tuning, I got these following values to have the best fit :

In [ ]:
model_dt = DecisionTreeClassifier(max_leaf_nodes=3,max_depth=4,random_state=42)

In [ ]:
model_dt.fit(train_df[numeric_cols], train_df[target_col])

DecisionTreeClassifier(max_depth=4, max_leaf_nodes=3, random_state=42)

The remaining part of the code is similar to that of logistic regression. We are again creating a list of Decision Tree models with different batches of validation data and reporting it's average accuracy.

In [ ]:
dt_models = []
train_avg=[]
val_avg=[]
for train_idxs, val_idxs in kfold.split(train_df[numeric_cols]):
    X_train, train_targets = train_df[numeric_cols].iloc[train_idxs], train_df[target_col].iloc[train_idxs]
    X_val, val_targets = train_df[numeric_cols].iloc[val_idxs], train_df[target_col].iloc[val_idxs]
    model, train_rmse, val_rmse = train_and_evaluate(X_train, 
                                                     train_targets, 
                                                     X_val, 
                                                     val_targets,model_dt
                                                     )
    dt_models.append(model)
    train_avg.append(train_rmse)
    val_avg.append(val_rmse)
    print('Train Accuracy: {}, Validation Accuracy: {}'.format(train_rmse, val_rmse))

print(f'Avg. Train Accuracy : {100*np.mean(train_avg):.3f} % , Avg. Val Accuracy : {100*np.mean(val_avg):.3f} %')

Train Accuracy: 0.9605695869508024, Validation Accuracy: 0.9598789792159957
Train Accuracy: 0.960635359116022, Validation Accuracy: 0.9596158905551171
Train Accuracy: 0.9594021310181531, Validation Accuracy: 0.964548802946593
Train Accuracy: 0.9601585109181794, Validation Accuracy: 0.9615232833464877
Train Accuracy: 0.9613917390160485, Validation Accuracy: 0.9565903709550119
Avg. Train Accuracy : 96.043 % , Avg. Val Accuracy : 96.043 %


We then use the same `predict_avg()` function to obtain predictions.




In [ ]:
preds_dt = predict_avg(dt_models, test_df[numeric_cols])

We can see that Decision Tree method seems to give us better results compared to Logistic Regression. Let us try further with Random Forests.



### 3. Random Forest

The last model that we try to use is Random Forests. The instance `RandomForestClassifier` is imported below.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

By similar hyperparameter tuning as done previously, these are the values that seem to get the highest validation accuracy:

In [ ]:
model_rf = RandomForestClassifier(min_samples_split=20, max_depth=4 ,n_jobs=-1, random_state=42,max_features='sqrt')

In [ ]:
model_rf.fit(train_df[numeric_cols], train_df[target_col])

RandomForestClassifier(max_depth=4, max_features='sqrt', min_samples_split=20,
                       n_jobs=-1, random_state=42)

Again, this looping statement below is similar to those above. It appends different models to the `rf_models` list, and reports training and validation accuracy using the `train_and_evaluate` function.

In [ ]:
rf_models = []
train_avg=[]
val_avg=[]
for train_idxs, val_idxs in kfold.split(train_df[numeric_cols]):
    X_train, train_targets = train_df[numeric_cols].iloc[train_idxs], train_df[target_col].iloc[train_idxs]
    X_val, val_targets = train_df[numeric_cols].iloc[val_idxs], train_df[target_col].iloc[val_idxs]
    model, train_rmse, val_rmse = train_and_evaluate(X_train, 
                                                     train_targets, 
                                                     X_val, 
                                                     val_targets,model_rf
                                                     )
    rf_models.append(model)
    train_avg.append(train_rmse)
    val_avg.append(val_rmse)
    print('Train Accuracy: {}, Validation Accuracy: {}'.format(train_rmse, val_rmse))

print(f'Avg. Train Accuracy : {100*np.mean(train_avg):.3f} % , Avg. Val Accuracy : {100*np.mean(val_avg):.3f} %')

Train Accuracy: 0.9605695869508024, Validation Accuracy: 0.9598789792159957
Train Accuracy: 0.960635359116022, Validation Accuracy: 0.9596158905551171
Train Accuracy: 0.9594021310181531, Validation Accuracy: 0.964548802946593
Train Accuracy: 0.9601585109181794, Validation Accuracy: 0.9615232833464877
Train Accuracy: 0.9613917390160485, Validation Accuracy: 0.9565903709550119
Avg. Train Accuracy : 96.043 % , Avg. Val Accuracy : 96.043 %


We then use the same `predict_avg()` function to obtain predictions.

In [ ]:
preds_rf = predict_avg(rf_models, test_df[numeric_cols])

Even after fine hyperparameter tuning, Random Forest seems to give the same result as Decision Tree, hence we can stop here. We will now update the test dataframe with our predictions.

Below is the code to update the value in the sample submission file.

In [ ]:
test_df[target_col]=preds_rf

Storing the test targets into our submission file's column.

In [ ]:
bench_df[target_col]=test_df[target_col]

In [ ]:
bench_df[target_col]=bench_df[target_col].astype(int)

These steps below is just used to download the final submission *csv* file. It is not necessary to execute this part of the notebook.

In [ ]:
from google.colab import files

In [ ]:
bench_df.to_csv('submissionf.csv',index=False)
files.download('submissionf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Summary 

We start by downloading the dataset from kaggle. This is done using the `opendatasets` library. We then store it in a dataframe using `pandas` and then inspect the data. The columns represent the features that we need to use to train the model, and we drop the column 'ID' since it doesn't help us. 

Next we preprocess the data and check for missing values, if any. Since there wasn't any missing data, the next step is to normalize it. This is done to maintaini general distribution as well as a ratio in data. Further, it also improves the performance and accuracy of machine learning models using various techniques and algorithms.

The next step is to build our models, and here we have used 3 models - Logistic Regression, Decision Tree and Random Forests. I have also tuned the hyperparameters to get the best fit of training and validation accuracy. 

We have K-Fold cross validation technique for validation. What this does is create a split in the initial training and uses different 'splits' as validation with the remaining as training. Further information on the same can be found here : [here](https://vitalflux.com/k-fold-cross-validation-python-example/)

And finally we use these models to predict our test data. It is to be noted that both RandomForests and Decision Tree gave the same result. However, they were better than the Logistic Regression model. The same can be verified by observing the validation accuracy reported above.
